In [1]:
!pip install openai

In [2]:
import numpy as np
import time
import os
import openai
import base64
import requests
import pandas as pd
from io import StringIO

# write the API key here
# openai.api_key = ...



In [3]:
task = "In the list below, you can find a list of jokes. I want you to select only those ones where the joke is based on a word that has these characteristics:\
\
- In the context of the joke, the word takes two different meanings.\
- It is spelled exactly in same way in both meanings: discard jokes base on homophones.\
- It's a one single word - no composite words.\
- It's a word present in the dictionary.\
\
For each sentence, then I need the you create two serious (without humor) sentences with the word at the center of the joke: one where the word takes the first meaning, and one where it takes the second meaning.\
\
The result of the computation must be returned in a format like this, to fit a CSV file separated by a semicolon:\
\
sentence; word; explanation; first new sentence; second new sentence\
\
\
"

column_names = ["sentence", "word", "explanation", "first_new_sentence", "second_new_sentence"]
df_new_selected_jokes_with_sentences = pd.DataFrame(columns=column_names)

In [4]:
folder = "only_true_jokes_CSVs"
numer_of_only_true_jokes=100000

for k in range (87000,100000,100):
    filename = folder+"/only_true_"+str(k)+"_"+str(k+100)+".csv"
    jokes = pd.read_csv(filename)

    jokes_list=""
    for i in range (len(jokes.text)):
        jokes_list=jokes_list+"\r "+jokes.text[i]

    messages=[]
    
    #openai headers
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
    
    #preparing the messages with the image and the prompt
    prompt = task+jokes_list
    print()
    print ("=== prompt")
    print (prompt)
    
    messages.append(
          {
            "role": "user",
            "content": [
              {"type": "text", "text": prompt
                  },
    
            ],
          }
        )
     
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
      )
    
    answer = response.choices[0].message.content
    
    print ("=== answer")
    print (answer)

    lines=answer.splitlines()
    cleaned_list = [item for item in lines if item.strip()]
    cleaned_list = [item for item in cleaned_list if item.strip("```")]
    cleaned_list = [row for row in cleaned_list if "```" not in row]
    cleaned_list = [s.replace("'", "").replace('"', "") for s in cleaned_list]
    cleaned_list = cleaned_list[1:]
    rows = [row.split(";") for row in cleaned_list]
    try:
        df_new_selected_jokes_with_sentences = pd.DataFrame(rows, columns=column_names)
        filename = folder+"/selection/selected_jokes_"+str(k)+"_"+str(k+100)+".csv"
        df_new_selected_jokes_with_sentences.to_csv(filename, encoding='utf-8', index=False)
    except:
        # Handle the error and continue
        #print(f"Failed to write {filename}: {e}")
        continue



=== prompt
 Remember, when someone calls you mean... just tell them that you prefer the term average.l.death?.. is based on a word that has these characteristics:- In the context of the joke, the word takes two different meanings.- It is spelled exactly in same way in both meanings: discard jokes base on homophones.- It's a one single word - no composite words.- It's a word present in the dictionary.For each sentence, then I need the you create two serious (without humor) sentences with the word at the center of the joke: one where the word takes the first meaning, and one where it takes the second meaning.The result of the computation must be returned in a format like this, to fit a CSV file separated by a semicolon:sentence; word; explanation; first new sentence; second new sentence
=== answer
Here's the filtered list of jokes according to your specified criteria:

```csv
sentence; word; explanation; first new sentence; second new sentence
How is prison like quidditch? the game ends

In [5]:
df_new_selected_jokes_with_sentences

,sentence,word,explanation,first_new_sentence,second_new_sentence
0,What do you call a woman who can balance 5 pin...,beatrix,A beatrix can refer to a persons name. In lit...,Beatrix was known for her graceful movements ...,The character Beatrix in the novel showcased ...
1,What does the dentist of the year get? a littl...,plaque,Plaque refers to a sticky film of bacteria th...,Dental plaque can lead to tooth decay if not ...,The plaque awarded to the employee recognized...
2,Whats the reverse of a blindfold? a burka.,burka,A burka is a garment worn by some Muslim wome...,The burka is significant in discussions aroun...,The burka serves as a means of visibility in ...
3,What do you call a gay paraplegic? rolaids.,rolaids,Rolaids is a brand of antacid used to relieve...,Rolaids can help soothe an upset stomach afte...,The relief provided by Rolaids can make a sig...
4,Note: The above conversion assumes some interp...,None,None,None,None
